## Multiclass TensoFlow Model Assessment with Lens
This notebook builds and assessed a digit-classification model with TensorFlow and Lens

In [1]:
%load_ext autoreload
%autoreload 2

### TensorFlow

In [2]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
print("TensorFlow version:", tf.__version__)
 

TensorFlow version: 2.10.0


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
from credoai.artifacts import KerasData

In [5]:
my_data = KerasData('keras', x_train, y_train)

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

model.run_eagerly = False

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.__class__.__module__

'keras.engine.sequential'

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train, epochs=3)

Train on 60000 samples
Epoch 1/3
 4992/60000 [=>............................] - ETA: 1s - loss: 0.8465 - accuracy: 0.7476

2022-12-07 14:26:06.584779: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-07 14:26:06.588121: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


60000/60000 [==============================] - 1s 17us/sample - loss: 0.3010 - accuracy: 0.9127
Epoch 2/3
60000/60000 [==============================] - 1s 14us/sample - loss: 0.1435 - accuracy: 0.9572
Epoch 3/3
60000/60000 [==============================] - 1s 14us/sample - loss: 0.1078 - accuracy: 0.9671


In [11]:
y_prob = model.predict(x_test)
np.argmax(y_prob, axis=1)

/Users/eli/anaconda3/envs/dev_gini/lib/python3.10/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


array([7, 2, 1, ..., 4, 5, 6])

In [12]:
from credoai.artifacts import KerasClassificationModel

In [13]:
my_model = KerasClassificationModel("keras_test", model)

In [14]:
my_model.predict(my_data.X)

array([5, 0, 4, ..., 5, 6, 8])

### Lens

In [19]:
from credoai.lens import Lens
from credoai.evaluators import Performance, DataProfiler

In [26]:
lens = Lens(
    model=my_model,
    assessment_data=my_data,
)

In [27]:
metrics = ['accuracy_score']
lens.add(Performance(metrics=metrics))

2022-12-07 14:29:08,605 - lens - INFO - Evaluator Performance added to pipeline. 


In [28]:
lens.add(DataProfiler())  # errors out

IndexError: list index out of range

In [22]:
lens.run()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [25]:
lens.get_results()[0]['results'][1]

,true_label,predicted_label,value
0,0,0,0.990883
1,1,0,0.000148
2,2,0,0.001511
3,3,0,0.000163
4,4,0,0.000514
...,...,...,...
95,5,9,0.002029
96,6,9,0.000000
97,7,9,0.001277
98,8,9,0.001367
